In [1]:
# Install necessary tools

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# Load a sample dataset (simulate recyclable items - use 'rock_paper_scissors' as proxy)
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Resize & Normalize
IMG_SIZE = 160
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
    return image, label

BATCH_SIZE = 32
ds_train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


c:\Users\ADMIN\Desktop\Software\AI_Software Engineering\Week6\Code\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 100%|██████████| 2/2 [02:17<00:00, 68.79s/ url]


Dataset rock_paper_scissors downloaded and prepared to C:\Users\ADMIN\tensorflow_datasets\rock_paper_scissors\3.0.0. Subsequent calls will reuse this data.


In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: rock, paper, scissors (simulated for recyclable items)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(ds_train, validation_data=ds_val, epochs=5)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 415ms/step - accuracy: 0.7115 - loss: 0.7050 - val_accuracy: 0.9762 - val_loss: 0.1091
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 27s 427ms/step - accuracy: 0.9910 - loss: 0.0793 - val_accuracy: 0.9921 - val_loss: 0.0559
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 28s 438ms/step - accuracy: 0.9959 - loss: 0.0410 - val_accuracy: 1.0000 - val_loss: 0.0356
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 28s 441ms/step - accuracy: 0.9986 - loss: 0.0263 - val_accuracy: 1.0000 - val_loss: 0.0253
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 28s 437ms/step - accuracy: 0.9999 - loss: 0.0185 - val_accuracy: 1.0000 - val_loss: 0.0192


In [3]:
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.2f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 443ms/step - accuracy: 1.0000 - loss: 0.0177
Test Accuracy: 1.00


In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save TFLite model
with open('recyclable_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpw33fswyl\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpw33fswyl\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmpw33fswyl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2249097815216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249097820672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249097821552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249097809760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249097817504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2248920996800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249097814160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249097815744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249097813632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2249098019584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [6]:
# Load interpreter
interpreter = tf.lite.Interpreter(model_path="recyclable_classifier.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a test image
for image, label in ds_test.take(1):
    test_image = image[0:1]

# Run inference
interpreter.set_tensor(input_details[0]['index'], test_image.numpy())
interpreter.invoke()
predictions = interpreter.get_tensor(output_details[0]['index'])
print("Predicted class:", np.argmax(predictions))


Predicted class: 1


In [7]:
with open("recyclable_classifier.tflite", "wb") as f:
    f.write(tflite_model)
